In [4]:
import json
#100x100 pix 

f = open('level_2.json', 'r')
data = json.load(f)
layerData = data['layers'][1:]
new_dict = {}
platformLocations = [] #platformlocation is missing the part #, only has 5 argument right now

######################################################
# json layout 
# background : {}
# side : {
#         platforms: {}
#         instances: {}
#        }
# objectives: {
#              key: {}
#              exitdoor: {}
#             }
#####################################################

#background json might need to be added 
sides = [0]*6 # 6 sides 
#one big json 
new_dict = {}
instances = []
mirror = {}
properties = {}
vanity = {}
light = {}
mechanism = {}
#another part of the json 
objective = {}
key_dict = {}
door = {}

# Add properties to instances 
for i in layerData: 
    side = int(i["name"])

    ratio = 100/640
    x = 640*side
    if(side == 0 or side == 1 or side == 2 or side == 3):
        y = 1280 
        #formula for x and y 
    elif(side == 4): # top
        x = 640 
        y = 640 
        #formula for x and y 
    elif(side == 5): # bottom
        x = 640
        y = 1920 

    objects = i["layers"]
    for each in objects:
        #Data = i["layers"][0]
        title = each["name"]
        if(title == "platforms"):
            # get the array of platforms and parse it to the array 
            platforms = each["objects"]
            print(len(platforms))
            for each1 in platforms: 
                one = []
                one.append((each1['x']-x+16)*ratio-50)
                one.append(abs(each1['y']-y+16)*ratio-50)
                one.append(each1['height']*ratio)
                one.append(each1['width']*ratio)
                one.append(side)
                #append the side # 
                parts = each1['properties'][0]
                parts_num = parts['value']
                one.append(parts_num)
                platformLocations.append(one)
                # need to add parts 
        #there is always ONE vanity 
        if(title == "Vanity"):
            vanity_obj = each["objects"][0]
            vanity["x"] = (vanity_obj['x']-x)*ratio-50
            vanity["y"] = (abs(vanity_obj['y']-y)*ratio-50)
            vanity["type"] = vanity_obj['type']
            vanity["texture"] = "cube/instances/"+vanity_obj['name']+".png"
            
            with open("vp.json", "w") as vpJson:
                json.dump(properties, vpJson)
            open_prop = open('vp.json')
            vp_prop = json.load(open_prop)

            vanity["properties"] = vp_prop
            
            with open("vanity.json", "w") as vanityJson:
                json.dump(vanity, vanityJson)
            open_vanityData = open('vanity.json')
            vanityData = json.load(open_vanityData)

            # add this to instance
            instances.append(vanityData)

        if(title == "Mirror"): 
            mirrors = each["objects"]
            for eachM in mirrors: 
                mirror["x"] = ((eachM['x']-x+16)*ratio-50)
                mirror["y"] = (abs(eachM['y']-y+16)*ratio-50)
                mirror["type"] = eachM['type']
                mirror["texture"] = "mirror/"+eachM['name']+".png"
                proper = eachM['properties']
                for eachp in proper: 
                    if (eachp['name'] == "movable"):
                        properties["movable"] = eachp['value']
                    if (eachp['name'] == "rotatable"): 
                        properties["rotatable"] = eachp['value']
                    if (eachp['name'] == "reflectionAngle"):
                        properties["reflectionAngle"] = eachp['value']
                    if (eachp['name'] == "type"):
                        properties["type"] = eachp['value']
                properties["orientation"] = eachM['rotation']

                with open("mp.json", "w") as mirror_propertyJson:
                    json.dump(properties, mirror_propertyJson)
                open_prop = open('mp.json')
                mirror_prop = json.load(open_prop)

                mirror["properties"] = mirror_prop

                with open("eachMirror.json", "w") as mirrorJson:
                    json.dump(mirror, mirrorJson)
                open_mirrorData = open('eachMirror.json')
                mirrorData = json.load(open_mirrorData)

                #add this to instance
                instances.append(mirrorData)
                #clear all the mirror data before it goes to next mirror
                mirror.clear()
                properties.clear()

        #POTENTIAL MULTIPLE LIGHT SOURCE - CAN BE UPDATED LATER
        if(title == "Light"): 
            crystal = each["objects"][0] 
            light["x"] = ((crystal['x']-x+20)*ratio-50)
            light["y"] = (abs(crystal['y']-y+21.5)*ratio-50)
            light["type"] = crystal['type']
            light['texture'] = crystal['name']+".png"

            proper = crystal['properties'][0]
            if(proper['value'] == 90):
                rotate = 1
            elif(proper['value'] == 180):
                rotate = 2 
            elif(proper['value'] == 270):
                rotate = 3
            else:
                rotate = 0

            properties["orientation"] = rotate 
            properties["side"] = side

            with open("property.json", "w") as propJson:
                json.dump(properties, propJson)
            open_prop = open('property.json')
            prop = json.load(open_prop)

            light["properties"] = prop

            with open("light.json", "w") as lightJson:
                json.dump(light, lightJson)
            open_lightData = open('light.json')

            lightData = json.load(open_lightData)
            instances.append(lightData)
            #clear light data incase there are multiple
            light.clear()
            properties.clear()

        if(title == "Key"): 
            keyhole = each["objects"][0]
            #ALWAYS ONE KEYHOLE 
            key_dict["x"] = ((keyhole['x']-x+16)*ratio-50)
            key_dict["y"] = (abs(keyhole['y']-y+16)*ratio-50)
            key_dict["side"] = side
            with open("keyhole.json", "w") as keyJson:
                json.dump(key_dict, keyJson)
            open_keyData = open('keyhole.json')
            keyData = json.load(open_keyData)
            objective["keyhole"] = keyData

        if(title == "door"):
            exit_door = each["objects"][0]
            door["x"] = ((exit_door['x']-x+16)*ratio-50)
            door["y"] = (abs(exit_door['y']-y+16)*ratio-50)
            door["side"] = side
            with open("door.json", "w") as doorJson:
                json.dump(door, doorJson)
            open_doorData = open('door.json')
            doorData = json.load(open_doorData)
            objective["exit_door"] = doorData

    new_dict["platformLocations"] = platformLocations
    new_dict["instances"] = instances
    with open("side.json", "w") as sideJson:
        json.dump(new_dict, sideJson)
    opening = open('side.json')
    # returns JSON object as 
    # a dictionary
    sideData = json.load(opening)
    sides[side] = sideData
    #print(sides[side])
    #clear the previous side data in that dictionary as it goes through the next one
    platformLocations.clear()
    instances.clear()
    new_dict.clear()

#print(new_dict)
# print(key_dict)
# print(objective)

new_dict["sides"] = sides
# read the objective json
with open("objective.json", "w") as objectiveJson:
    json.dump(objective, objectiveJson)
open_objectiveData = open('objective.json')
objectiveData = json.load(open_objectiveData)
new_dict["objectives"] = objectiveData

with open('levelTesting.json', 'w') as demo: 
    json.dump(new_dict, demo, indent = 4)

# x = open('levelTesting.json', 'r')
# data1 = json.load(x)
# data2 = data1['platformLocations']
# print(data2)
# for each in data2:
#     print(each[0])

FileNotFoundError: [Errno 2] No such file or directory: 'level_2.json'